# Лабораторная работа №2
## Выполнил студент группы БФИ2001 Семёнов Андрей Владиславович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)

### Задание №1
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [1]:
from __future__ import annotations
from typing import Any, Generator, Optional, Union
from random import randint
from time import time

In [2]:
class AbstractSearcher:
    def __init__(self, a: list[int]) -> None:
        self.a = a[:]

    def search(self, n: int) -> int:
        raise NotImplementedError("This method must be implemented")

    def put(self, n: int) -> None:
        i: int = self.search(n)
        self.a.insert(i, n)

    def remove(self, n: int) -> int:
        i: int = self.search(n)
        self.a.pop(i)
        return n
# #### Бинарный поиск
class BinarySearcher(AbstractSearcher):
    def search(self, n: int) -> int:
        l: int = 0
        r: int = len(self.a) - 1
        while l <= r:
            mid = l + (r-l) // 2
            if self.a[mid] < n:
                l = mid + 1
            elif self.a[mid] > n:
                r = mid - 1
            else:
                return mid

        raise Exception(f"No such elem found {n}")

In [3]:
# #### Бинарное дерево
class BinaryTreeSearcher(AbstractSearcher):
    def __init__(self, a: list[int]) -> None:
        self.a = a[:]
        self.b = BinaryTree(self.a[0])
        for i in a[1:]:
            self.b.insert(BinaryTree(i))

    def search(self, n: int) -> int:
        res = self.b.search(n, self.b)
        if res is not None:
            return res.data
        return -1

    def put(self, n: int) -> None:
        self.b.insert(BinaryTree(n))

    def remove(self, n: int) -> int:
        return self.b.remove(n)

In [4]:
class BinaryTree:

    def __init__(self, data: int, parent = None) -> None:
        self.left: Union[None, BinaryTree] = None
        self.right: Union[None, BinaryTree] = None
        self.parent: Union[None, BinaryTree] = parent
        self.data: int = data

    def search(self, n: int, t: Optional[BinaryTree]) -> Optional[BinaryTree]:
        while t is not None and n != t.data:
            if n > t.data:
                t = t.right
            else:
                t = t.left

        return t

    def insert(self, z: BinaryTree) -> None:
        y = None
        x = self

        while x is not None:
            y = x
            if z.data < x.data:
                x = x.left
            else:
                x = x.right

        z.parent = y
        if y is None:
            self = z
        elif z.data < y.data:
            y.left = z
        else:
            y.right = z

    def remove(self, n: int) -> int:
        z: Optional[BinaryTree] = self.search(n, self)
        if z is None:
            raise Exception("No such element")

        if z.left is None and z.right is not None:
            self.transplant(z, z.right)
        elif z.right is None and z.left is not None:
            self.transplant(z, z.left)
        elif z.right is not None and z.left is not None:
            y = self.tree_minimum(z.right)
            if y.parent != z and y.right is not None:
                self.transplant(y, y.right)
                y.right = z.right
                y.right.parent = y

            self.transplant(z, y)
            y.left = z.left
            y.left.parent = y

        return -1

    def transplant(self, u: BinaryTree, v: BinaryTree) -> None:
        if u.parent is None:
            self = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v

        if v is not None:
            v.parent = u.parent

    def tree_minimum(self, t: BinaryTree) -> BinaryTree:
        while t.left is not None:
            t = t.left

        return t

    def inorder_tree_walk(self, t: Optional[BinaryTree]) -> None:
        if t is not None:
            self.inorder_tree_walk(t.left)
            print(t.data)
            self.inorder_tree_walk(t.right)

In [5]:
# #### Метод Фибоначчи
class FibonacciSearcher(AbstractSearcher):
    def search(self, n: int) -> int:
        len_a: int = len(self.a)
        start: int = -1

        f0 = 0
        f1 = 1
        f2 = 1
        while(f2 < len_a):
            f0 = f1
            f1 = f2
            f2 = f1 + f0


        while(f2 > 1):
            mid: int = min(start + f0, len_a - 1)

            if n > a[mid]:
                f2 = f1
                f1 = f0
                f0 = f2 - f1
                start = mid

            elif n < a[mid]:
                f2 = f0
                f1 = f1 - f0
                f0 = f2 - f1

            else:
                return mid

        raise Exception(f"No such elem found {n}")

In [6]:
# #### Интерполяционный
class InterpolationSearcher(AbstractSearcher):
    def search(self, n: int) -> int:
        left: int = 0
        right: int = len(a) - 1

        while a[left] < n and a[right] > n:
            mid: int = int(left + (n - a[left]) * (right - left) / (a[right] - a[left]))
            if n > a[mid]:
                left = mid + 1
            elif n < a[mid]:
                right = mid - 1
            else:
                return mid

        if a[left] == n:
            return left
        elif a[right] == n:
            return right

        return -1

In [7]:
def test_searcher(a: list[int], searcher: AbstractSearcher) -> float:
    times: list[float] = []

    for elem in a:
        start: float = time()
        res: int = searcher.search(elem)
        times.append(time() - start)
        assert res != -1

    return sum(times) / len(times)

In [8]:
a: list[int] = [randint(-1000, 1000) for _ in range(10 ** 4)]
a.sort()

In [9]:
print("Testing binary search")
binary_searcher: AbstractSearcher = BinarySearcher(a)
binary_res: float = test_searcher(a, binary_searcher)
print(f"Binary search tests passed, average time: {binary_res} ms")

Testing binary search
Binary search tests passed, average time: 3.3317327499389647e-06 ms


In [10]:
print("Testing binary tree search")
binary_tree_searcher: AbstractSearcher = BinaryTreeSearcher(a)
binary_tree_res: float = test_searcher(a, binary_searcher)
print(f"Binary tree search tests passed, average time: {binary_tree_res} ms")

Testing binary tree search
Binary tree search tests passed, average time: 3.2415390014648438e-06 ms


In [11]:
print("Testing fibonacci search")
fib_searcher: AbstractSearcher = FibonacciSearcher(a)
fib_res: float = test_searcher(a, fib_searcher)
print(f"Fibonacci search tests passed, average time: {fib_res} ms")

Testing fibonacci search
Fibonacci search tests passed, average time: 6.322956085205078e-06 ms


In [12]:
print("Testing interpolation search")
interpolation_searcher: AbstractSearcher = InterpolationSearcher(a)
inte_res: float = test_searcher(a, interpolation_searcher)
print(f"Interpolation search tests passed, average time: {inte_res} ms")

Testing interpolation search
Interpolation search tests passed, average time: 2.043747901916504e-06 ms


### Задание №2
Написать соответствующие решения для заданных методов.

#### Простое рехэширование

In [13]:
class HashTable:
    def __init__(self, n: int = 5):
        self.size = 0
        self.slots: list[Any] = [None] * n
        self.data: list[Any] = [None] * n
        self.q: int = 0

    def __setitem__(self, key, value) -> None:
        hashvalue: int = self._hash(key)

        if self.slots[hashvalue] is None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = value

        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = value
            else:
                nextslot: int = self._rehash(hashvalue)
                self.q += 1

                while self.slots[nextslot] is not None \
                        and self.slots[nextslot] != key:
                    nextslot = self._rehash(nextslot)
                    self.q += 1

                if self.slots[nextslot] is None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = value
                else:
                    self.data[nextslot] = value
        self.q = 0

    def __getitem__(self, key: int) -> Any:
        startslot: int = self._hash(key)
        position: int = startslot

        while self.slots[position] is not None:
            if self.slots[position] == key:
                self.q = 0
                return self.data[position]
            else:
                position = self._rehash(position)
                self.q += 1
                if position == startslot:
                    continue

        self.q = 0
        raise KeyError(f"No key {key}")

    def _hash(self, key: int) -> int:
        return key % len(self.slots)

    def _rehash(self, oldhash: int) -> int:
        return (oldhash + 1) % len(self.slots)

#### Рехэширование с помощью псевдослучайных чисел

In [14]:
class RandomHashTable(HashTable):
    def __init__(self, n: int = 5):
        super().__init__(n)
        self.randints: list[int] = [randint(0, 1000) for _ in range(n)]

    def _rehash(self, oldhash: int) -> int:
        if self.q == len(self.randints):
            self.randints.extend([randint(0, 100) for _ in range(self.q)])

        newhash: int = (oldhash + self.randints[self.q]) % len(self.slots)
        return newhash

#### Метод цепочек

In [15]:
class ListNode:
    def __init__(self, key, val):
        self.pair = (key, val)
        self.next: Union[None, ListNode] = None

In [16]:
class ChainedHashTable:

    def __init__(self):
        self.m = 1000;
        self.h: list[Union[None, ListNode]] = [None]*self.m

    def __setitem__(self, key, value):
        index = self._hash(key)

        cur = self.h[index]
        if cur is None:
            self.h[index] = ListNode(key, value)

        else:
            while True:
                if cur.pair[0] == key:
                    cur.pair = (key, value)
                    return

                if cur.next == None: break
                cur = cur.next

            cur.next = ListNode(key, value)

    def __getitem__(self, key):
        index = self._hash(key)
        cur = self.h[index]

        while cur:
            if cur.pair[0] == key:
                return cur.pair[1]
            else:
                cur = cur.next
        raise KeyError(f"No such key: {key}")

    def _hash(self, key) -> int:
        return key % self.m

In [17]:
def test_hashing(hashable) -> None:
    tests_dict: dict[int, str] = dict({
        54: "cat",
        26: "dog",
        44: "lion",
        15: "tiger",
        20: "duck"
    })

    for k, v in tests_dict.items():
        hashable[k] = v

    for k, v in tests_dict.items():
        assert hashable[k] == v

In [18]:
print("Testing hash table")
test_hashing(HashTable())
print("Hash table tests passed")

Testing hash table
Hash table tests passed


In [19]:
print("Testing random hash table")
test_hashing(RandomHashTable())
print("Random hash table tests passed")

Testing random hash table
Random hash table tests passed


In [20]:
print("Testing chained hash table")
test_hashing(ChainedHashTable())
print("Chained hash table tests passed")

Testing chained hash table
Chained hash table tests passed


### Задание №3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.

In [21]:
class Solution:
    def __init__(self, board: list[list[int]]) -> None:
        self.board = board
        self.steps: int = 0

    def search(self, col: int = 0) -> bool:
        self.steps += 1

        if col == 7:
            return True

        for row in range(8):
            if self._can_place(row, col):
                self.board[row][col] = 1

                if self.search(col + 1):
                    return True

                self.board[row][col] = 0

        return False

    def _can_place(self, row: int, col: int) -> bool:
        iter_up: Generator = (i for i in range(row+1, 8))
        iter_down: Generator = (i for i in range(row-1, -1, -1))

        for i in range(col-1, -1, -1):
            if self.board[row][i]:
                return False;

            up = next(iter_up, None)
            if up is not None and self.board[up][i]:
                return False;

            down = next(iter_down, None)
            if down is not None and self.board[down][i]:
                return False;

        return True

In [22]:
board: list[list[int]] = [
    [0 for _ in range(8)]
    for _ in range(8)
]

In [23]:
chess = Solution(board)

In [24]:
if chess.search():
    print("Solution found")
    for row in board:
        print(row)
else:
    print("Solution not found")

Solution found
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]


In [25]:
print(f"{chess.steps=}")

chess.steps=11
